<a href="https://colab.research.google.com/github/tmkhang1999/Frutome/blob/main/Frutome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests_html

In [39]:
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import pandas as pd
import re

session = HTMLSession()

def make_clickable(val):
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

def scrape_google(keyword, length, language='en'):
    # Set up the search URL
    url = f"https://www.google.com/search?q={keyword}&num={length}&hl={language}"
    res = session.get(url)
    
    # Parse the HTML response with BeautifulSoup
    soup = BeautifulSoup(res.text, 'html.parser')
    
    # Extract the search result items
    search_results = soup.find_all('div', {'class': 'kvH3mc'})
    
    # Create an empty Pandas dataframe to store the results
    results_df = pd.DataFrame(columns=['Link', 'Title', 'Description', 'Bold'])
    
    # Loop through the search results and extract the relevant information
    for res in search_results:
        # Extract the title, link, text, and bold text
        title = res.select_one('h3').get_text() if res.select_one('h3') else None
        link = res.select_one('.yuRUbf a')['href'] if res.select_one('.yuRUbf a') else None
        des = res.select_one('.VwiC3b').get_text() if res.select_one('.VwiC3b') else None
        bold = res.select_one('.VwiC3b span em').get_text() if res.select_one('.VwiC3b span em') else None

        # Add the result to the Pandas dataframe
        row_dict = pd.DataFrame({'Link': [link], 'Title': [title], 'Description': [des], 'Bold': [bold]})
        results_df = pd.concat((results_df, row_dict), ignore_index=True)

    # make clickable hyperlinks and style the link column
    results_df = results_df.style.format({'Link': make_clickable})
    results_df.set_properties(subset=['Link'], **{'font-size': '11px', 'word-wrap': 'break-word', 'max-width': '200px'})
  
    # Return the Pandas dataframe
    return results_df

In [40]:
df = scrape_google('rebate realtor in denver', 10)

In [41]:
df

,Link,Title,Description,Bold
0,https://www.prevu.com/blog/commission-rebate-denver,What are Buyer Commission Rebates in Denver?,"Oct 8, 2022 — Prevu offers the largest commission rebate in Colorado, allowing buyers to receive a homebuyer rebate of up to 2% of the sale price of a home.",rebate
1,https://www.hqhomesales.com/home-buyer-commission-rebate-denver-colorado,"Denver, CO Home Buyer 50% Commission Rebate | HQ Homes",HQ Homes Flat Fee Realty offers home buyers a generous Buyers Agent Commission Rebate of up to 50% of the buyers agent commission (this equals up to 1.5% of the ...,Homes
2,https://www.houzeo.com/blog/discount-real-estate-brokers-in-denver/,"Best Discount Real Estate Brokers in Denver, CO","Feb 27, 2023 — Rankings: Best Discount Real Estate Brokers in Denver, CO ; 1. Houzeo, $349. Flat Fee ; 2. Fixed Rate Real Estate, $5000. Flat Fee ; 3. HQ Homes, 1 ...",Discount Real Estate Brokers in Denver
3,https://www.discount-realtor.com/co/denver/,Real Estate Discounts & Rebates | Denver DISCOUNT ...,Denver discount real estate brokers & agents! Buyer rebates! Experienced! Full service! Put more $ in YOUR pocket! Since 1999! US-wide! DISCOUNT-REALTOR®,Denver discount real estate brokers
4,https://www.caare.org/buyeragentsrebate/colorado/,Colorado – Buyer Agents Who Pay Rebates,We will help you purchase the home you want and then provide 50% of our commission back to you as a rebate after the closing. You find the home. We close the ...,rebate
5,https://www.flatfeegroup.com/Colorado/Denver/buyerrebatequestions.php,Buyer Rebate Questions - Denver,"When a seller lists their home for sale with a Realtor, they usually agree to pay 6% (commission varies) at closing. Half or typically 3% goes to the listing ...",Realtor
6,https://www.upnest.com/1/post/who-are-the-discount-realtors-in-denver/,Who are the Discount Realtors in Denver?,That commission is actually due to the listing broker. The individual Denver real estate agent cannot directly accept any money in their name. Their brokerage ...,Denver real estate agent
7,https://www.upnest.com/re/home-buyer-rebates-states/co,Colorado Buyer Rebates | Save Money When Buying in ...,"When you buy with UpNest, you'll receive a $150 Amazon e-card plus a commission rebate from your agent where applicable, often 20-30% of the agent commission.",rebate
8,https://coloradocommissionrebate.com/,Colorado Commission Rebate: Looking to buy a home ...,Colorado Commission Rebate is the best Colorado based real estate agents since 2009. We help you Buy your new home at discounted prices by offering you a ...,Rebate
9,https://michaelborgelt.com/denver-real-estate-commission-refund/,1/2 Back – Denver Real Estate - Michael Borgelt,"Home Buyers. The Old Way When you buy a home the seller is responsible for paying your realtor. But, that fee is baked in the purchase price, ...",realtor
